In [5]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import cohen_kappa_score


def load_all_dataframes(base_folder: Path) -> pd.DataFrame:
    splits = ["train", "valid", "test"]

    df_train, df_valid, df_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for split in splits:
        df = pd.read_csv(base_folder / f"{split}.csv")
        if split == "train":
            df_train = df
        elif split == "valid":
            df_valid = df
        else:
            df_test = df

    return pd.concat([df_train, df_valid, df_test])


REPO_ROOT = Path(".")

adam_df = load_all_dataframes(REPO_ROOT / "src/sec_certs/data/reference_annotations/adam")
jano_df = load_all_dataframes(REPO_ROOT / "src/sec_certs/data/reference_annotations/jano")
agreement_series = adam_df.label == jano_df.label

print(f"Cohen's Kappa: {cohen_kappa_score(adam_df.label, jano_df.label)}")
print(f"Percentage agreement: {agreement_series.loc[agreement_series == True].count() / agreement_series.count()}")


Cohen's Kappa: 0.7101271765978729
Percentage agreement: 0.8225
